# Frozen Lake Example for Policy Iteration
![policy_iteration](../images/lecture_1/policy_iteration.png)

In [1]:
import gymnasium as gym
import numpy as np
from typing import List
env=gym.make("FrozenLake-v1",map_name="4x4",is_slippery=True)

## Check the Attributes in the Env

In [2]:
# transition dict P P[state][action]->(probability, next_state, reward, is_terminated)
env.P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [3]:
# number of actions and number of states
print("number of actions: %d" % env.action_space.n)
print("number of states: %d" % env.observation_space.n)

number of actions: 4
number of states: 16


## State-Action Q

In [4]:
def q_state_action(v: np.ndarray, state: int, action: int, P, gamma):
    q = 0
    for (probability, s_next, reward, _) in P[state][action]:
        q += probability*(reward+gamma*v[s_next])
    return q


## Policy Evaluation

In [5]:
from typing import List


def policy_evaluation(policy: List, num_states, P, gamma, epsilon=1e-5):
    v_prev = np.zeros(num_states)
    while True:
        v_next = np.array([q_state_action(
            v_prev, s_i, policy[s_i], P, gamma) for s_i in range(num_states)])
        delta_v = np.max(np.power(v_next-v_prev, 2))
        if delta_v < epsilon:
            break
        else:
            v_prev = v_next
    return v_prev

policy_evaluation([0]*env.observation_space.n,env.observation_space.n,env.P,0.9)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

## Policy Improvement

In [6]:
def policy_improvement(v: np.ndarray, policy: List, num_states: int, num_actions: int, P, gamma):
    new_policy = policy.copy()
    is_policy_stable = True
    for s_i in range(num_states):
        qs = np.array([q_state_action(v, s_i, action, P, gamma)
                      for action in range(num_actions)])
        idx_qsmax = np.argmax(qs)
        if idx_qsmax != policy[s_i]:
            new_policy[s_i] = idx_qsmax
            is_policy_stable = False
    return new_policy, is_policy_stable


## Policy Iteration (Solve the MDP)

In [7]:
def policy_iteration(num_states: int, num_actions: int, P, gamma=0.9, epsilon=1e-5):
    policy = [0]*num_states
    v = policy_evaluation(
        policy, num_states, P, gamma, epsilon)
    while True:
        policy, is_policy_stable = policy_improvement(
            v, policy, num_states, num_actions, P, gamma)
        if is_policy_stable:
            break
        else:
            v = policy_evaluation(policy, num_states, P, gamma, epsilon)
    return policy


In [8]:
policy=policy_iteration(env.observation_space.n, env.action_space.n, env.P)


## Test the Selected Policy

In [9]:
def policy_test(env, policy: List, num_test: int):
    state = env.reset()[0]
    total_reward = 0
    for _ in range(num_test):
        while True:
            next_state, reward, is_terminated, truncated, info = env.step(
                policy[state])
            state = next_state
            if is_terminated:
                total_reward += reward
                state = env.reset()[0]
                break
    print("won %d of %d games" % (total_reward, num_test))

policy_test(env,policy,1000)

won 787 of 1000 games
